# 📚 Introduction to Agents with LangGraph

In this notebook you'll learn how to create Agents using LangGraph.

We will use an LLM to answer questions, and execute actions for us!

## ⚙️ Setup

👉 Run the cell below to import a couple of basic libraries.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from pprint import pprint
from IPython.display import Markdown

For this challenge you'll need your API key again.

👉 Run the cell below to load it.

In [ ]:
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file

## 🤖 Why Agents?

Remember how we used an LLM with tool calling (aka function calling)?

First, we gave the LLM information about the tools / functions we have. Then, we asked the LLM natural language questions and it told us which tool it would use, and it gave us the arguments we'd need to use to cool the tool.

That was already pretty cool, but it left the actual calling of the tool up to us.

With agents, we'll have the LLM decide which tool to use and with arguments, and the agent will execute the work using the tools.

## 🦾 Context: our own PA

In this challenge, we'll create our own PA, our Personal Agent.

The agent will be able to help us with a couple of our routine daily tasks:

- Update us on our stock portfolio.
- Get recipes (can't live of stock prices, you got to eat too).
- Retrieve information from Wikipediat (to feed your brain).

There are many frameworks to create agents.

In this notebook we'll use LangGraph, from the creators of LangChain 🦜🔗. It will interact nicely with the tools we've already seen before.

## 🧠 Setup our brain

In it's most simple form, an agent consist of a brain (an LLM), an agent executor, and different tools.

Before we start setting up the tools, let's set up our brain. We can do this like before, using LangChain. 

👉 Set up a model. This time use the Gemini 2.5 Flash model, not the Lite version because it tends not to work very well with our use case.

In [ ]:
# YOUR CODE HERE

## 📈 A first tool: check up on our favourite stocks

Wouldn't it be great if we could ask our PA in natural language about our stocks?

For this, our agent needs to be able to retrieve stock prices. Back in the data sourcing unit, we used the **Massive API** (formerly known as Polyon) to get stock prices. We spent a fair bit of time to use their API and source data from it.

It turns out they have built a tool that we can plug straight into our agent. Under the hood it's using the same API, but it's completely ready for an agentic workflow.

### Authenticate with the Massive API

Before we can use, we'll have to authenticate though. Head to the [massive.com](https://massive.com/dashboard) website to get your API key. (Normally you should have already set up an account and an API key in the data sourcing unit. If not, create one. Note: Massive was previously known as Polygon, so you might have stored your password under that name.)

1. Copy your API key from the website.
1. Open `.env` file in this unit's folder (one level up from your challenge folder).
1. Write a new line: `POLYGON_API_KEY=your_massive_api_key` in the file.
1. Save and close the file.
1. Run the cell below to reload the environment variables.

> Why `POLYGON_API_KEY` if the API is called Massive? The LangChain tools haven't changed to the new name yet, and require an environment variable named `POLYGON_API_KEY`.

In [ ]:
load_dotenv()
'POLYGON_API_KEY' in os.environ

### Load the Polygon tools

With that done, let's load the Polygon tools. 

> The LangChain tools are still named Polygon at the time of writing. If you can't find Polygon, try Massive instead.

LangChain has quite a big [library of tools](https://docs.langchain.com/oss/python/integrations/tools) that we can use.

Have a look at the library and find the Polygon tool documentation.

👉 Find in the documentation how you can load the tools. What is the output of the `.get_tools()` method?

In [ ]:
# YOUR CODE HERE

Polygon gives us 4 tools.

Throughout this challenge we'll create a `tools` list that will contain all the tools our agent can use.

👉 Create a list with the 4 Polygon tools to start with.

In [ ]:
# YOUR CODE HERE

### Create an agent to use the tools

Go and check the [LangChain quick start](https://docs.langchain.com/oss/python/langchain/quickstart) to see how you can create an agent. Just a basic agent for now.

👉 Create an agent and name it `agent_executor`. It should use the model and tools we created before:

In [ ]:
# YOUR CODE HERE

Let's try it out.

👉 Ask the agent for a random stock's price. You can use the company name, it will probably figure out the ticker for you.

In [ ]:
from langchain_core.messages import HumanMessage

query = "Give me Apple's stock price?"
response = agent_executor.invoke(
    {"messages": [HumanMessage(content=query)]}
)
response["messages"][-1].content

Well, that went wrong very quickly. What happened? Have a look at the end of the error message: it tells us that our free plan doesn't have the required accesses. It turns out our LLM was using the `polygon_last_quote` tool, which asks for current prices. We can only look at historical prices.

We don't want our agent to completely crash for such a reason. So, let's add some error handling. Again, the [LangChain documentation](https://docs.langchain.com/oss/python/langchain/agents#tool-error-handling) shows us how to do this. It's not the core of what we want to do in this challenge, so here's the code you can use:

In [ ]:
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"],
        )

agent_executor = create_agent(model, tools, middleware=[handle_tool_errors])

What we are doing here is adding some so called "middleware" that will catch errors from our tools, and relay that information to the model, instead of letting the whole thing crash.

Let's use our improved model:

In [ ]:
query = "Give me Apple's stock price?"
response = agent_executor.invoke({"messages": [HumanMessage(content=query)]})
response["messages"][-1].content

That's much better, isn't it?

Let's have a look at what our agent has been doing:

While you're building your agent, it's better to not just show the last message, but also the intermediate steps.

One way to do that is to look at the complete response instead, but that's not very user-friendly as you will see:

In [ ]:
response

Also, with this approach, we'd have to wait until the agent has finished all the steps before we see the result. If you have many tools and steps, that could take long.

So let's put it in streaming mode to display the results as they are generated:

In [ ]:
for step in agent_executor.stream({
    "messages": [
        HumanMessage(content=query)
    ]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

That's more interesting: now we can see the **Tool Message**: the output of the tool as it is handed over to the model by the error handler.

And it tells us that our free plan doesn't have the required accesses. It turns out our LLM was using the `polygon_last_quote` tool, which asks for current prices. We can only look at historical prices.

👉 Try asking the agent for the **closing** price on a specific date (you can write the date like you want, just make sure it has day, month and year, and that it's was a working day).

In [ ]:
query = "Give me Apple's closing price on the 5th of May 2025."
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

That's much better. The good thing is that we can also verify the intermediate steps: we can see which tool was used, which arguments were used, and what values it returned. This way we can verify that the LLM we use isn't hallucinating.

👉 Now ask the tool for "yesterday's" closing price:

In [ ]:
query = "Give me yesterday's closing price of Apple."
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

Bummer, it doesn't know what date we are. That makes sense: after all, based on its training data, the LLM can't know what date we are. Let's solve that.

## 📆 Adding the date

To tell the LLM what date we are, we can give it a simple tool.

👉 Create your own custom tool `get_today`.

Step-by-step instructions:
- Start from the basic function below.
- Convert it into a LangChain tool (like we did in the tool calling challenge).
- Add a docstring to the function so the agent knows what the tool is supposed to do.

In [ ]:
from datetime import datetime

# TODO: Convert this function to a tool
def get_today() -> str:
    # TODO: Add a docstring
    return datetime.today().strftime("%Y-%m-%d")

Then add your `get_today` tool to the list of tools the agent can use. You'll have to reinstantiate your agent with the updated list. It doesn't update automatically.

In [ ]:
# YOUR CODE HERE

In [ ]:
tools

Ask your agent again for yesterday's closing price. You should see it calling the `get_today` function to figure out the date. Based on that it should then call the Polygon API for yesterday's price and return it. (Assuming that yesterday the stock exchange was open... In that case, ask it for the "last working day's closing price" instead.)

In [ ]:
query = "Give me the last working day's closing price of Apple."
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## 📝 The memory problem, and how to solve it

If you have been lucky, the agent so far always gave you a response without a follow up question.
If it would have asked you for extra input, how would you be able to answer it? 

You could give it a new prompt. That assumes the agent still remembers the previous prompts and all of its answers. 

Does it? Let's try a simple experiment.

👉 Prompt the agent twice:
1. Say hi, and tell it your name.
1. Then ask it for your name.

In [ ]:
# Use the agent
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Hi, I'm Jules! and I live in Brussels.")]}
)
response["messages"][-1].content

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What's my name again?")]}
)
response["messages"][-1].content

Your answer may vary, but it won't tell you your name. That's because your agent has no memory.

To make it reuse earlier parts of our conversation, we'd have to feed it the whole message history when we prompt it with our next prompt. That's why `messages` in the `invoke` method is a list: the whole conversation history.

So, we'd have to maintain a list with all the message, and each time append our new question and the new answer to it. That's very tedious. And we're only having one conversation now. In reality we'd have to manage multiple chats at the same time.

Fortunately LangGraph helps us to do that with **memory**, or **checkpointers**.

If your memory doesn't betray you, you already know what we'll tell you next:

👉 Go check the [LangGraph documentation](https://python.langchain.com/docs/tutorials/agents/) and add memory to your agent. Then try it with two separate prompts again. (Don't forget to add a `thread_id` in the `config`.)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_agent(model, tools, middleware=[handle_tool_errors], checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Hi, I'm Jules! and I live in Brussels.")]},
    config=config
)
response["messages"][-1].content

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What's my name again?")]},
    config=config,
)
response["messages"][-1].content

Now we are also able to answer the agent's follow-up questions.

Try it out:

In [ ]:
query = "Give me Apple's closing price."
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
query = "Last Friday's"
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
query = "And Amazon's?"
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

That starts to look like something.

By combining tools to get stock prices and today's date with memory, the agent starts to work pretty well.

But we've been careful in our prompts: we always asked it for the closing price. If we had asked for the current price, it would still fail.

In [ ]:
query = "Give me Meta's current price."
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

How could we solve that?

We can give the agent a bit more guidance. It should not use the current price API endpoint if it's not authorized too, but fall back to the historical prices (i.e. the last closing price).

We are using a pre-built tool, so we can't change the tool itself. But we can do someting else...

## 🗒️ Adding a system prompt

So far we didn't provide any system prompt to our agent. And in case you wondered: no, LangGraph hasn't added a default one for us either.

We could go and search for an existing one in the [LangChain Hub](https://smith.langchain.com/hub), but let's build one ourselves for our specific use case.

👉 Write a `system_prompt` and add it to your model. The prompt should instruct the agent to fall back on historical prices if the API doesn't allow to get current prices.

In [ ]:
# YOUR CODE HERE

Test your agent with the new prompt (notice how we changed the `thread_id` to start a new conversation - just to be sure the agent isn't influenced by our previous question):

In [ ]:
query = "Give me Meta's current price."
config = {"configurable": {"thread_id": "abc126"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

### 🚀 Congratulations! You know have a working agent.

Let's look back at what we did to get this working:

🧠 We instantiated an LLM. The brain of our agent.

🧰 We created a toolbox for our agent, with:

   - 📈 The Polygon tool to get stock prices

   - 🗓️ A simple custom made tool to get today's date

📝 We added memory to our agent, so we could have an interactive conversation with it.

🗒️ We added a system prompt to give it more guidance.

## 🏭 Putting your agent in production

You probably already noticed that using the agent like this in the notebook is a bit clumsy.

In this part, we'll turn our agent into a real application. We'll make a CLI tool that you can interact with in the terminal (CLI = Command Line Interface).

Open `my_pa.py` and investigate it. We have added some basic functionality to it. Up to you to make it run the agent we created in this notebook.

When you run the file through `python my_pa.py`, the `main()` function will start an (almost infinite) loop. At each iteration it:
- Asks the user for a new prompt
- Uses the agent with the new prompt
- Outputs the agent's response
- And starts all over

👉 Your task is to fill in the gaps:
- Add the necessary imports
- Instantiate the tools (ignore the Wikipedia and recipe tools for now, we'll add them later)
- Instantiate the LLM and the agent
- Complete the `use_agent()` function 

Everything is already in the notebook. It's mainly a question of copying the good bits into the `.py` file and make it all work together.

Once you're done with that, come back to this notebook. We'll add a couple more tools to our agent.

## 🍰 Adding our own recipes tool

In the tool calling challenge, we created a tool to retrieve recipes. Let's add that to our agent.

We included the tool in `recipe.py`. Have a look at it. Notice that we included the `@tool` directly in here.

We just need to import it, and check that it's a tool:

In [ ]:
from recipe import get_recipes
type(get_recipes)

👉 Give your agent access to this new tool:

In [ ]:
# YOUR CODE HERE

Try it out:

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="Give me recipes with chicken.")]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="Give me the first recipe's instructions.")]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

That last one is not very useful. It would be handy if we'd actually get the recipe itself, no?

We can do that with the `RequestsGetTool`:

In [ ]:
from langchain_community.tools.requests.tool import RequestsGetTool
from langchain_community.utilities.requests import TextRequestsWrapper

requests_tool = RequestsGetTool(
    requests_wrapper=TextRequestsWrapper(headers={}), allow_dangerous_requests=True
)

🚧 See that `allow_dangerous_requests` argument? We need to set it explicitly to make our requests work.

⚠️ It also means it's **dangerous**: our agent can now make any kind of GET request it wants. This could do unintended things, and could be abused by end users of your agent. If you are deploying your agent to let other people use it, you definitely do not want to include this.

👉 We're only making a Personal Agent for ourselves, so let's add it to the agent:

In [ ]:
# YOUR CODE HERE

Try the new model, and ask it for the recipe. You might have to nudge it a bit in the good direction with your prompts.

Our agent now can fetch information from our recipes site, but it can also make requests to other sites or APIs. That's why it's dangerous. Ask it to get information from your favourite news site.

Sometimes it will be able to extract the information from the sites, sometimes it will only be able to give you the pure HTML. There are better [tools](https://python.langchain.com/docs/integrations/tools/#web-browsing) than Requests for that, but you'd have to sign up for them. That's why we stuck to a very basic tool for this demonstration.

## 📚 Extending the agent's knowledge base

The LLM only knows what it learned during training. But we could easily extend its knowledge with the Wikipedia tool.

Add [Wikipedia](https://python.langchain.com/docs/integrations/tools/wikipedia/) to your agent's tool box, and try it out.

In [ ]:
# YOUR CODE HERE

Try your agent and ask it about something that happened recently, and see what tool it uses.

In [ ]:
# YOUR CODE HERE

## 🏭 Add the new tools to `my_pa.py`

After experimenting in the notebook with new tools, bring them over to `my_pa.py`.

## 🏁 Congratulations! You made a fully working agent.

You learned how to:

🦾 Create an agent , starting from an LLM 🧠

📝 Add memory to the agent to allow interactive conversations

🧰 Give it diffent tools, both existing ones and self-built custom tools

🗒️ Tweak its behaviour using a system prompt

## 🚀 Want to take it further?

Here are some ideas you can work with:

- Can you make the agent to always respond in your own language?
- Make it a bit more easy to work with the recipes?
- What other tools could you add for your daily tasks?

Don't forget to commit and push all your work!